In [56]:
import random

cities = {
    'A': {'days': 3, 'cost': 80},
    'B': {'days': 4, 'cost': 85},
    'C': {'days': 5, 'cost': 100},
    'D': {'days': 2, 'cost': 45},
    'E': {'days': 3, 'cost': 75},
    'F': {'days': 1, 'cost': 25},
    'G': {'days': 6, 'cost': 50},
    'H': {'days': 8, 'cost': 205},
    'I': {'days': 1, 'cost': 24},
    'J': {'days': 7, 'cost': 200},
}

def generate_population(population_size, max_cities, max_days):
    population = []
    # ทำการสุ่ม population เริ่มต้น
    for _ in range(population_size):
        # จำนวนเมืองที่จะสุ่ม ณ ตอนนั้น ซึ่งคิดว่าสุ่มตั้งแต่ 2 - max_cities จะคุ้มและเร็วที่สุด
        num_cities = random.randint(2, max_cities)
        curr_cities = []
        total_days = 0
        while total_days < max_days and len(curr_cities) < num_cities:
            # สุ่มเมืองที่จะไป
            city = random.choice(list(cities.keys()))
            if city in [c for c, _ in curr_cities]:
                continue
            days_to_stay = cities[city]['days']
            if total_days + days_to_stay > max_days:
                continue
            curr_cities.append((city, days_to_stay))
            total_days += days_to_stay
        population.append(curr_cities)
    return population

def fitness_function(select_cities):
    # คำนวณค่า fitness โดยคำนวณจาก จำนวนวันที่เข้าเมือง / ราคาทั้งหมด เพราะคิดว่าน่าจะคิดได้ง่าย
    total_days = 0
    total_cost = 0
    for city, days_to_stay in select_cities:
        total_days += days_to_stay
        total_cost += cities[city]['cost']
    if total_cost > 0:
        return total_days / total_cost
    return 0

def fitness_calculate(population):
    fitness_values = []
    for select_cities in population:
        fitness_values.append(fitness_function(select_cities))
    return fitness_values

# สำหรับเลือก Parent ในแต่ละรอบ
def roulette_wheel_selection(population, fitness_values, num_parents):
    parents = []
    total_fitness = sum(fitness_values)
    
    # Normalize ให้ค่า fitness อยู่ในช่วง 0-1 สำหรับค่าความน่าจะเป็น
    probs = []
    for value in fitness_values:
        probs.append(value / total_fitness)

    for _ in range(num_parents):
        index = random.choices(range(len(population)), weights=probs)[0]
        parents.append(population[index])

    return parents

# Crossover
def crossover(parent1, parent2):
    child1, child2 = [], []
    crossover_point = random.randint(1, min(len(parent1), len(parent2)) - 1)
    child1.extend(parent1[:crossover_point] + parent2[crossover_point:])
    child2.extend(parent2[:crossover_point] + parent1[crossover_point:])
    return child1, child2

# Mutation
def mutation(child, max_days):
    # สุ่มเมืองที่จะเข้าไปแทนที่เมืองเดิม
    index = random.randint(0, len(child) - 1)
    while True:
        new_city = random.choice(list(cities.keys()))
        if new_city in [c for c, _ in child]:
            continue
        days_to_stay = cities[new_city]['days']
        break
    # ถ้าเมืองที่สุ่มมา สามารถเข้าไปได้โดยไม่เกินจำนวนวันที่กำหนด
    total_days = sum([day for _, day in child])
    if total_days - child[index][1] + days_to_stay <= max_days:
        child[index] = (new_city, days_to_stay)
    return child

In [77]:
population_size = 10
max_cities = 5
max_days = 14
epochs = 20

# Phase 1: Generate the initial population
init_population = generate_population(population_size, max_cities, max_days)
print(init_population)

iteration_population = init_population

# Phase 2: Selection for breeding offsprings (We will loop here until optimal solution)

for _ in range(epochs):
    # Individuals test for fitness
    fitness_values = fitness_calculate(iteration_population)

    # Selection for reproduction

    num_parents = 2  # Number of parents to select for crossover
    selected_parents = roulette_wheel_selection(iteration_population, fitness_values, num_parents)
    print("Selected Parents:")
    for parent in selected_parents:
        print(parent)

    # Reproduction (Crossover & Mutation)
    offspring = []
    for i in range(0, 2):
        parent1 = selected_parents[0]
        parent2 = selected_parents[1]
        child1, child2 = crossover(parent1, parent2)
        offspring.append(mutation(child1, max_days))
        offspring.append(mutation(child2, max_days))
    
    iteration_population = offspring

# Phase 3 : Evaluation of a population
final_fitness_values = fitness_calculate(iteration_population)
print(final_fitness_values)

[[('G', 6), ('F', 1), ('I', 1), ('D', 2), ('B', 4)], [('G', 6), ('H', 8)], [('H', 8), ('E', 3), ('F', 1), ('I', 1)], [('J', 7), ('E', 3), ('I', 1), ('F', 1), ('D', 2)], [('E', 3), ('G', 6)], [('H', 8), ('I', 1), ('E', 3)], [('D', 2), ('F', 1), ('I', 1)], [('J', 7), ('C', 5), ('I', 1), ('F', 1)], [('I', 1), ('J', 7), ('D', 2), ('F', 1)], [('A', 3), ('F', 1), ('I', 1), ('E', 3), ('C', 5)]]
Selected Parents:
[('G', 6), ('F', 1), ('I', 1), ('D', 2), ('B', 4)]
[('D', 2), ('F', 1), ('I', 1)]
Selected Parents:
[('D', 2), ('F', 1), ('I', 1), ('D', 2), ('B', 4)]
[('D', 2), ('F', 1), ('I', 1), ('D', 2), ('B', 4)]
Selected Parents:
[('A', 3), ('F', 1), ('I', 1), ('D', 2), ('B', 4)]
[('A', 3), ('F', 1), ('I', 1), ('D', 2), ('B', 4)]
Selected Parents:
[('G', 6), ('F', 1), ('I', 1), ('D', 2), ('B', 4)]
[('A', 3), ('E', 3), ('I', 1), ('D', 2), ('B', 4)]
Selected Parents:
[('A', 3), ('E', 3), ('I', 1), ('D', 2), ('B', 4)]
[('G', 6), ('F', 1), ('I', 1), ('D', 2), ('B', 4)]
Selected Parents:
[('A', 3), 

In [86]:
# Final list

max_value = 0
final_index = 0
for index, value in enumerate(final_fitness_values):
    if value > max_value:
        max_value = value
        final_index = index

final_selected = iteration_population[final_index]

total_cost = 0
total_days = 0

print("Final Selected:")

for city, day in final_selected:
    total_cost += cities[city]['cost']
    total_days += day
    print(f"City: {city} - Days: {day}")

print("Total Cost: ", total_cost)
print("Total Days: ", total_days)

Final Selected:
City: F - Days: 1
City: I - Days: 1
City: E - Days: 3
City: C - Days: 5
City: D - Days: 2
Total Cost:  269
Total Days:  12
